In [ ]:
import os
import glob
import trackpy as tp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import curve_fit
import numpy as np
from matplotlib.ticker import ScalarFormatter
from matplotlib.collections import LineCollection

framerate = 4.37
microns_per_pixel = 0.072

In [ ]:
# Combines several trajectory or step files into one csv

file_directory = r"C:\Users\lizau\Desktop\walker_tracker_for_article\walker_tracker\example\analysing_rotated_trajectories" 
in_files = glob.glob(f'{file_directory}/*.csv')
out_dir = Path(rf'{file_directory}\traj_analysis_out')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

trays_to_combine = []
max_particle_id = 0

for file in in_files:
    tray = pd.read_csv(file)
    if 'particle' in tray.columns:         # Offset the particle IDs to ensure uniqueness
        tray['particle'] += max_particle_id
        max_particle_id = tray['particle'].max() + 1
    trays_to_combine.append(tray)
combined_df = pd.concat(trays_to_combine, ignore_index=True)
combined_df = combined_df.sort_values(by='frame').reset_index(drop=True)
combined_df.to_csv(f'{out_dir}/combined_sorted_particles.csv')

print("DataFrames combined successfully!")

In [ ]:
# If analysing the steps: calculates average step size
average_step_len = np.average(combined_df['step_len'])
print(f'Average step size: {average_step_len*72} +- {np.std(combined_df["step_len"]*72)}')

# If analysing an axis-aligned steps file (tracks aligned to one of the axes):
average_step_len_dx = np.average(combined_df['dx'])
average_step_len_dy = np.average(combined_df['dy'])

In [ ]:
# Plots coordinates (user-defined --> both on the same plot or seperately) vs time
    # This notebook only makes sense if analysing pre-rotated movies, where tracks have been aligned along one of the axes

tray_len_query = 0 #change if you only want to plot coordinates of particles with trajectories longer than tray_len_query
both_coord = True #change to False if you only want one coordinate plotted
coordinate = 'x' #which coordinate to plot?


particles_trays_filt_plot = combined_df.query(f'length>{tray_len_query}') 

# Get unique particle IDs 
particles_to_plot = particles_trays_filt_plot['particle'].unique()

# Which dimension are you plotting?
plotted_x, plotted_y = False, False #leave as is

# Create the plot

plt.figure(figsize=(12, 6))
norm_coord_y = []
for pid in particles_to_plot:
    particle_data = particles_trays_filt_plot[particles_trays_filt_plot['particle'] == pid]
    if particle_data.empty:
        continue
    if both_coord == True:
        start_y = particle_data['y'].iloc[0]
        normalized_y = particle_data['y'] - start_y
        norm_coord_y += [normalized_y]
        start_x = particle_data['x'].iloc[0]
        normalized_x = particle_data['x'] - start_x
        start_frame = particle_data['frame'].iloc[0]
        normalized_frame = particle_data['frame'] - start_frame
        if not plotted_x:
            plt.plot(normalized_frame * 1/framerate, normalized_x * microns_per_pixel*1000, color='#1F77B4', linewidth=2.5, alpha=0.6, label='perpendicular to track axis')
            plotted_x = True
        else:
            plt.plot(normalized_frame * 1/framerate, normalized_x * microns_per_pixel*1000, color='#1F77B4', linewidth=2.5, alpha=0.6)

        if not plotted_y:
            plt.plot(normalized_frame * 1/framerate, normalized_y * microns_per_pixel*1000, color='#FF7F0E', linewidth=2.5, alpha=0.4, label='along the track axis')
            plotted_y = True
        else:
            plt.plot(normalized_frame * 1/framerate, normalized_y * microns_per_pixel*1000, color='#FF7F0E', linewidth=2.5, alpha=0.4)
    else:
        start_coord = particle_data[f'{coordinate}'].iloc[0]  # First y-coordinate
        normalized_coord = particle_data[f'{coordinate}'] - start_coord
        start_frame = particle_data['frame'].iloc[0]
        normalized_frame = particle_data['frame'] - start_frame
        if coordinate == 'x':
            start_y = particle_data['y'].iloc[0]
            normalized_y = particle_data['y'] - start_y
            norm_coord_y += [normalized_y]
        else:
            norm_coord_y += [normalized_coord]
        plt.plot(normalized_frame*0.25, normalized_coord*72, label=f'Particle {pid}')

# Customize plot
plt.xlabel('time [s]', fontsize=16)
if both_coord == True:
    plt.ylabel('position [nm]', fontsize=16)
else:
    plt.ylabel(f'{coordinate} position [nm]', fontsize=16)
plt.ylim(((np.min(np.concatenate(norm_coord_y))-0.2)*72, (np.max(np.concatenate(norm_coord_y))+0.2)*72))
if both_coord == True:
    plt.title(f'X and Y Positions vs. Frame for Particles with Trajectory Length > {tray_len_query}')
else:
    plt.title(f'{coordinate} Position vs. Frame for Particles with Trajectory Length > {tray_len_query}')
plt.grid(True)
plt.tight_layout()
plt.legend(fontsize=16, loc='upper right')
if both_coord == True:
    plt.savefig(f'{out_dir}/Normalized_xy_positions_vs_time__tray_len_{tray_len_query}.png')
else:
    plt.savefig(f'{out_dir}/Normalized_{coordinate}_vs_time_frame__tray_len_{tray_len_query}.png')
plt.show()



In [ ]:
# Plots histrograms of maximal trajectory span for both coordinates on the same plot

tray_len_query = 0 # only particles with trajectories longer than tray_len_query will be analysed
tray_offsets_x = []
tray_offsets_y = []

particles_trays_filt_hist = combined_df.query(f'length>{tray_len_query}')
particles_to_plot = particles_trays_filt_hist['particle'].unique()

for pid in particles_to_plot:
    particle_data = particles_trays_filt_hist[particles_trays_filt_hist['particle'] == pid]
    
    if particle_data.empty:
        continue

    offset_x = (np.max(particle_data['x']) - np.min(particle_data['x'])) * microns_per_pixel*1000
    offset_y = (np.max(particle_data['y']) - np.min(particle_data['y'])) * microns_per_pixel*1000
    
    tray_offsets_x.append(offset_x)
    tray_offsets_y.append(offset_y)

# Plot histograms
plt.figure(figsize=(6, 6))
plt.hist(tray_offsets_x, bins=10, alpha=0.8, label='trajectory span in X', color='#1F77B4')
plt.hist(tray_offsets_y, bins=10, alpha=0.8, label='trajectory span in Y', color='#FF7F0E')
plt.xticks(np.arange(0, plt.xlim()[1] + 10, 50))

plt.xlabel('trajectory span [nm]', fontsize=14)
plt.ylabel('count', fontsize=14)
#plt.title(f'Max X and Y Offsets for Trajectories > {tray_len_query} frames')
plt.legend(fontsize=12, loc='upper right')
plt.tick_params(axis='both', labelsize=14)
#plt.grid(True)
plt.tight_layout()

plt.savefig(f'{out_dir}/xy_max_offset_tray_len_{tray_len_query}_frame.png')
print(f'Mean X span: {np.mean(tray_offsets_x)} +- {np.std(tray_offsets_x)}')
print(f'Mean Y span: {np.mean(tray_offsets_y)} +- {np.std(tray_offsets_y)}')
plt.show()


In [ ]:

# Plots histograms of net movement (final - initial position) for both coordinates on the same plot

tray_len_query = 0 # only particles with trajectories longer than tray_len_query will be analysed

net_movements_x = []
net_movements_y = []

# Filter particles with sufficient trajectory length
particles_trays_filt_hist = combined_df.query(f'length>{tray_len_query}')
particles_to_plot = particles_trays_filt_hist['particle'].unique()

# Compute net movements
for pid in particles_to_plot:
    particle_data = particles_trays_filt_hist[particles_trays_filt_hist['particle'] == pid]

    if particle_data.empty:
        continue

    net_x = (particle_data['x'].iloc[-1] - particle_data['x'].iloc[0]) * microns_per_pixel*1000
    net_y = (particle_data['y'].iloc[-1] - particle_data['y'].iloc[0]) * microns_per_pixel*1000

    net_movements_x.append(net_x)
    net_movements_y.append(net_y)

# Plot histograms
plt.figure(figsize=(6, 6))
plt.hist(net_movements_x, bins=10, alpha=0.8, label='net movement in X', color='#1F77B4')
plt.hist(net_movements_y, bins=10, alpha=0.8, label='net movement in Y', color='#FF770E')

# Optional: set x-tick labels every 10 nm
#plt.xticks(np.arange(round(min(net_movements_x + net_movements_y) - 10, -2),
#                     round(max(net_movements_x + net_movements_y) + 10, -2), 100))
max_xy_combined = round(max(net_movements_x + net_movements_y) + 10, -2)
min_xy_combined = round(min(net_movements_x + net_movements_y) - 10, -2)
min_max_xy_combined = [max_xy_combined, np.absolute(min_xy_combined)]
print(-max(min_max_xy_combined), max(min_max_xy_combined))
plt.xticks(np.arange(-max(min_max_xy_combined), max(min_max_xy_combined) + 101, 100))

plt.xlabel('net movement [nm]', fontsize=14)
plt.ylabel('count', fontsize=14)
#plt.title(f'Net X and Y Movements for Trajectories > {tray_len_query} frames', fontsize=14)
plt.legend(fontsize=12, loc='upper right')
plt.tick_params(axis='both', labelsize=14)
plt.tight_layout()

plt.savefig(f'{out_dir}/xy_net_movement_tray_len_{tray_len_query}_frame.png')
plt.show()


In [ ]:
# Plots MSD for both dimensions and the overall msd

tray_len_query_msd = 25
max_lag = 25

formatter = ScalarFormatter()
formatter.set_scientific(True)
formatter.set_powerlimits((-2, 2))  # controls when sci notation kicks in

# Filter your data
particles_trays_filt_msd = combined_df.query(f'length>{tray_len_query_msd}')

# Compute EMSD for both x and y coordinates
em_xy = tp.emsd(particles_trays_filt_msd, microns_per_pixel, framerate, max_lagtime=max_lag)
em_x = tp.emsd(particles_trays_filt_msd, microns_per_pixel, framerate, max_lagtime=max_lag, pos_columns=['x'])
em_y = tp.emsd(particles_trays_filt_msd, microns_per_pixel, framerate, max_lagtime=max_lag, pos_columns=['y'])

# Plot both on the same graph
fig, ax = plt.subplots(figsize=(6,6))
ax.plot(em_xy.index, em_xy, "o", color='#7F7F7F', alpha=0.7, label='Overall displacement')
ax.plot(em_x.index, em_x, "o", color='#1F77B4', alpha=0.7, label='Perpendicular to track axis')  # blue line for x
ax.plot(em_y.index, em_y, "o", color='#FF7F0E', alpha=0.7, label='Along the track axis')  # red line for y
ax.set_ylabel(r"$\langle \Delta r^2 \rangle$ [$\mu m^2$]", fontsize=14)
ax.set_xlabel("Lag time $\mathit{t}$ [s]", fontsize=14)
yticks = np.linspace(0,
    #np.round(np.min([em_x.min(), em_y.min(), em_xy.min()]), 4),
    0.0035,
    8)

ax.set_yticks(yticks)
ax.yaxis.set_offset_position('left')
ax.yaxis.get_offset_text().set_fontsize(14)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis='both', labelsize=14)

handles, labels = ax.get_legend_handles_labels()

# Add the legend below the figure, centered
fig.legend(handles, labels, loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.05), fontsize=14)

# Adjust layout to make space for the legend
fig.tight_layout()
fig.subplots_adjust(bottom=0.15)

plt.savefig(f"{out_dir}/combined_trays_{tray_len_query_msd}_maxlag_{max_lag}_emsd_xy_x_y.png", bbox_inches='tight')

In [ ]:
# Functions that can be used for fitting emsd

def quadratic(x,a,b):
    return a*x + b*x**2

def cubic(x,a,b,c):
    return a*x + b*x**2 + c*x**3

def fourthOrder(x,a,b,c,d):
    return a*x + b*x**2 + c*x**3 + d*x**4

def fullEquation(x,v,tau,D):
    return 4*D*x + 2*(v**2)*(tau**2)*(x/tau + np.exp(-x/tau) - 1)

def linear(x,a, b):
    return a*x + b
    
def control(x,a):
    return a*x

def exponential(x,a,b):
    return np.exp(a)*x**b

def powerLaw(x,a,b):
    return a*x**b

def powerLaw2(x,a,b):
    return x**a
    
def exponential2(x,a,b):
    return a*np.exp(-x/b)

In [ ]:
# Fit a linear function to the linear part of the linearized emsd plot

linear_start = 0
linear_end = 10**(max_lag/4)

# Extract the specific part of the data for fitting
mask = (em_xy.index > linear_start) & (em_xy.index < linear_end)
x_linear = em_xy.index[mask]
y_linear = em_xy[mask]

# Fit the linear region with curve_fit
popt, pcov = curve_fit(linear, np.log10(x_linear), np.log10(y_linear))
diff_coeff = (10**popt[1]/4)*10**6

plt.plot(np.log10(em_xy.index), np.log10(em_xy), "o", color='#7F7F7F', alpha=0.7, label='Overall displacement')
plt.plot(np.log10(x_linear), linear(np.log10(x_linear), *popt), label=f"Linear Fit: y = {popt[0]:.2e}x")
plt.plot([], [], ' ', label=f"D = {np.round(diff_coeff)}"+r" nm$^2$/s")# + {np.round(popt[1],2)}")
plt.legend(fontsize=12)
plt.ylabel(r"log$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]", fontsize=14)
plt.xlabel("log(lag time) [s]", fontsize=14)
plt.savefig(f"{out_dir}\log_emsd_xy_linfit_maxlag_{max_lag}.png", bbox_inches='tight')
plt.show()

In [ ]:
# Finds particles with dwell times +- seconds

plus = 2
minus = 2
average_dwell_time = np.average(combined_df['length'])
max_length = np.max(combined_df['length'])
avg_particles = []
for particle_id in combined_df['particle'].unique():
    lengths = combined_df.loc[combined_df['particle'] == particle_id, 'length']
    if ((lengths > (average_dwell_time - minus)) & (lengths < (average_dwell_time + plus))).any():
        avg_particles += [particle_id]
        print(particle_id)

#print(f"particle with max length is: {combined_df.loc[combined_df['length'] == max_length, 'particle']}")

In [ ]:
# Plots a particle trajectory (y vs x) and colors it time dependantly

avg_particle = 206 #choose which particle's trajectory you want to plot

particle_df = combined_df[combined_df['particle'] == avg_particle]

# 2Extract x, y, and frame
x = particle_df['x'].to_numpy()
y = particle_df['y'].to_numpy()
frames = particle_df['frame'].to_numpy()
x = (x - np.mean(x))*72
y = -(y - np.mean(y))*72
time = (frames - frames[0])*0.25

points = np.array([x, y]).T.reshape(-1,1,2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

lc = LineCollection(segments, cmap='viridis', array=time, linewidth=2, alpha=0.8)
fig = plt.figure(figsize=(6,6))
plt.gca().add_collection(lc)
plt.gca().set_aspect('equal', adjustable='box')
plt.xticks(np.arange(-100, 101, 50))
plt.yticks(np.arange(-100, 101, 50))
plt.xlim(-125, 125)
plt.ylim(-125, 125)
cbar = fig.colorbar(lc, fraction=0.046, pad=0.04)
cbar.set_label(label='Time [s]', fontsize=18)
cbar.ax.tick_params(labelsize=18, width=1)
cbar.outline.set_linewidth(1)
plt.xlabel('x [nm]', fontsize = 18)
plt.ylabel('y [nm]', fontsize = 18)
plt.tick_params(axis='both', labelsize=18)
plt.savefig(f'{out_dir}\particle_{avg_particle}_trajectory.png')
plt.show()

